In [1]:
import sys
import subprocess
sys.path.append('f:/seismic/code/core')
sys.path.append('/seismic/code/core')
sys.path.append('/kaggle/input/my-seismic-library/')
import kaggle_support as kgs
kgs.preallocate_matrices=False
import importlib
import seis_model
import time
import pandas as pd
import numpy as np
#kgs.cache_dir_write = kgs.cache_dir_write+'/../cache_evaluate/'
#kgs.cache_dir_read = kgs.cache_dir_write

F:/seismic/models/brendan/*.pth
Loading:  F:/seismic/models/brendan\unet2d_caformer_seed3_epochbest.pt
n_models: 1


In [ ]:
# Set up model
importlib.reload(seis_model)
model = seis_model.default_model()
# model.models[1].model_FlatVel = seis_model.DummyModel()
# model.models[1].model_FlatVel.state=1
# model.models[1].model_Style_A = seis_model.DummyModel()
# model.models[1].model_Style_A.state=1
# model.models[1].model_TV2D = seis_model.DummyModel()
# model.models[1].model_TV2D.state=1
model.read_cache = False

In [ ]:
%%time
data = kgs.load_all_train_data(validation_only=True)[::50]
model.models[1].run_in_parallel = True
data_out = model.infer(data)

In [ ]:
dfTimings = pd.read_csv(kgs.timing_filename)
dfTimings = dfTimings.groupby('family')['time_taken'].mean().to_frame('time_taken')

In [ ]:
#kgs.disable_caching = True
families = list(set([d.family for d in data]))
families.sort()
res = dict()
res['family'], res['score_brendan'], res['score_mine'], res['score_mine_max'], res['time_taken'] = [],[],[],[],[]
for f in families:
    res['family'].append(f)
    this_data_out = []
    for d in data_out:
        if d.family==f:
            this_data_out.append(d)
    data_out_brendan = model.models[0].infer(this_data_out)
    res['score_brendan'].append(kgs.score_metric(data_out_brendan,show_diagnostics=False)[0])
    #data_out = model.infer(this_data[:1])
    t=time.time()
    try:
        res['time_taken'].append(dfTimings.loc[f].to_numpy()[0])
    except:
        res['time_taken'].append(0)
    score,_,res_all = kgs.score_metric(this_data_out,show_diagnostics=False)
    res['score_mine'].append(score)
    res['score_mine_max'].append(np.max(res_all))
    #print(pd.DataFrame(res))
df = pd.DataFrame(res)
print('')
print('')
print('Brendan local score: ', np.mean(df['score_brendan']))
print('My local score: ', np.mean(df['score_mine']))
print('Estimate public LB: ', 28.8*np.mean(df['score_mine'])/np.mean(df['score_brendan']))
df

In [ ]:
# No extra model: 28.8
# FlatVel:
# StyleA: 

In [ ]:
28-30*0.081